# Prova Mineração de Dados

### load libraries

In [ ]:
import pandas as pd
import transformers


### load data

In [ ]:
df = pd.read_csv('data/train.csv')

display(df.head())


In [ ]:

# pd dummies discourse_type column
df = pd.get_dummies(df, columns=['discourse_type'])

display(df.head())


In [ ]:

# X and y
X = df.drop(['discourse_id', 'essay_id', 'discourse_effectiveness'], axis=1)
y = df['discourse_effectiveness']

display(X.head())
display(y.head())


In [ ]:
# use transformer to create embedding
tokenizer = transformers.BertTokenizer.from_pretrained('bert-base-uncased')
X_embedded = tokenizer.batch_encode_plus(X['discourse_text'], max_length=512, padding=True, return_tensors='pt')

display(X_embedded['input_ids'][:5])


In [ ]:
# add embeddings to X
X['embeddings'] = X_embedded['input_ids'].squeeze().tolist()

display(X.head())


In [ ]:
from yellowbrick.text import TSNEVisualizer

tuple_vectors = X['embeddings'].values.tolist()

# Create the visualizer and draw the vectors
tsne = TSNEVisualizer()
tsne.fit(tuple_vectors)
tsne.poof()


In [ ]:
# train model
from sklearn.linear_model import LogisticRegression

X_embeddings = X['embeddings'].values.tolist()

model = LogisticRegression()
model.fit(X_embeddings, y)


In [ ]:
# score on train
score = model.score(X_embeddings, y)
print(f'Score on train: {score}')


In [ ]:
# score on test
df_test = pd.read_csv('data/test.csv')
df_test = pd.get_dummies(df_test, columns=['discourse_type'])

X_test = df_test.drop(['discourse_id', 'essay_id'], axis=1)
X_test['embeddings'] = X_embedded['input_ids'].squeeze().tolist()

X_test_embeddings = X_test['embeddings'].values.tolist()

score = model.score(X_test_embeddings, y)

print(f'Score on test: {score}')
